# LSTM Training

## Data and Library Import

In [1]:
from numpy import array
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import ipynb
import keras
from sklearn.metrics import mean_squared_error
import pandas as pd

## Get GME and AMC Datasets

In [2]:
#Store variables for training
%store -r AMC_Price
%store -r AMC_Open
%store -r AMC_High
%store -r AMC_Low
%store -r AMC_Vol
%store -r AMC_Change

%store -r GME_Price
%store -r GME_Open
%store -r GME_High
%store -r GME_Low
%store -r GME_Vol
%store -r GME_Change

## Sequence for Splitting Datsets to Fit LSTM

In [3]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## LSTM Prediction Model Training

In [4]:
def LSTM_Prediction_Model(train_price, train_open, train_high, train_low, train_volume, train_change): 
    data_price = train_price
    data_open = train_open
    data_high = train_high
    data_low = train_low
    data_vol = train_volume
    data_change = train_change

    # choose a number of time steps
    n_steps = 8
    # split into samples / reshape from x,y to x,y,z
    Xprice, y = split_sequence(data_price, n_steps)
    Xprice = Xprice.reshape((Xprice.shape[0], n_steps, 1))
    
    Xopen, y = split_sequence(data_open, n_steps)
    Xopen = Xopen.reshape((Xopen.shape[0], n_steps, 1))
    
    Xhigh, y = split_sequence(data_high, n_steps)
    Xhigh = Xhigh.reshape((Xhigh.shape[0], n_steps, 1))
    
    Xlow, y = split_sequence(data_low, n_steps)
    Xlow = Xlow.reshape((Xlow.shape[0], n_steps, 1))
    
    Xvol, y = split_sequence(data_vol, n_steps)
    Xvol = Xlow.reshape((Xvol.shape[0], n_steps, 1))
    
    Xchange, y = split_sequence(data_change, n_steps)
    Xchange = Xchange.reshape((Xchange.shape[0], n_steps, 1))
    
    #concatenate training data into one array (samples, steps, features)
    Xtraining = np.concatenate((Xprice, Xopen, Xhigh, Xlow, Xchange), axis = 2)

    n_filters=10
    fsize=5
    window_size=8
    n_features=5
    hour_filter = 8

    prediction_model = Sequential()
    layer0 = tf.keras.layers.BatchNormalization()
    layer1 = tf.keras.layers.Conv1D(n_filters, fsize, padding="same", activation="relu", input_shape=(window_size, n_features))
    layer2 = tf.keras.layers.Flatten()
    layer3 = tf.keras.layers.Dense(1000, activation='relu')
    layer4 = tf.keras.layers.Dense(100)
    layer5 = tf.keras.layers.Dense(1)
    prediction_model.add(layer0)
    prediction_model.add(layer1)
    prediction_model.add(layer2)
    prediction_model.add(layer3)
    prediction_model.add(layer4)
    prediction_model.add(layer5)
    prediction_model.compile(optimizer="adam", loss="mean_squared_error")
    prediction_model.fit(Xtraining, y, epochs=200, batch_size =32, verbose=1)
    return prediction_model

## Model Training and Saving

### Calling Training Function for Datasets

In [5]:
AMCModel = LSTM_Prediction_Model(AMC_Price, AMC_Open, AMC_High, AMC_Low, AMC_Vol, AMC_Change)
GMEModel = LSTM_Prediction_Model(GME_Price, GME_Open, GME_High, GME_Low, GME_Vol, GME_Change)

Epoch 1/200
52/52 [==============================] - 1s 1ms/step - loss: 0.2580
Epoch 2/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0021
Epoch 3/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0017
Epoch 4/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 5/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 6/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 7/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 8/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0011
Epoch 9/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0014
Epoch 10/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0015
Epoch 11/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 12/200
52/52 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 13/200
52/52 [=================

52/52 [==============================] - 0s 1ms/step - loss: 5.8141e-04
Epoch 99/200
52/52 [==============================] - 0s 1ms/step - loss: 5.2919e-04
Epoch 100/200
52/52 [==============================] - 0s 1ms/step - loss: 4.9160e-04
Epoch 101/200
52/52 [==============================] - 0s 1ms/step - loss: 4.5020e-04
Epoch 102/200
52/52 [==============================] - 0s 1ms/step - loss: 5.3539e-04
Epoch 103/200
52/52 [==============================] - 0s 1ms/step - loss: 4.5707e-04
Epoch 104/200
52/52 [==============================] - 0s 1ms/step - loss: 5.7282e-04
Epoch 105/200
52/52 [==============================] - 0s 1ms/step - loss: 4.2940e-04
Epoch 106/200
52/52 [==============================] - 0s 1ms/step - loss: 4.9823e-04
Epoch 107/200
52/52 [==============================] - 0s 1ms/step - loss: 5.9147e-04
Epoch 108/200
52/52 [==============================] - 0s 1ms/step - loss: 4.9195e-04
Epoch 109/200
52/52 [==============================] - 0s 1ms/step - 

52/52 [==============================] - 0s 1ms/step - loss: 2.0825e-04
Epoch 194/200
52/52 [==============================] - 0s 1ms/step - loss: 1.8112e-04
Epoch 195/200
52/52 [==============================] - 0s 1ms/step - loss: 2.3018e-04
Epoch 196/200
52/52 [==============================] - 0s 1ms/step - loss: 1.8512e-04
Epoch 197/200
52/52 [==============================] - 0s 1ms/step - loss: 2.3071e-04
Epoch 198/200
52/52 [==============================] - 0s 1ms/step - loss: 1.8330e-04
Epoch 199/200
52/52 [==============================] - 0s 1ms/step - loss: 2.2970e-04
Epoch 200/200
52/52 [==============================] - 0s 1ms/step - loss: 1.9822e-04
Epoch 1/200
75/75 [==============================] - 0s 1ms/step - loss: 0.2361
Epoch 2/200
75/75 [==============================] - 0s 1ms/step - loss: 0.0012
Epoch 3/200
75/75 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 4/200
75/75 [==============================] - 0s 1ms/step - loss: 9.8796e-04
Ep

75/75 [==============================] - 0s 1ms/step - loss: 5.3268e-04
Epoch 91/200
75/75 [==============================] - 0s 1ms/step - loss: 5.7102e-04
Epoch 92/200
75/75 [==============================] - 0s 1ms/step - loss: 4.7849e-04
Epoch 93/200
75/75 [==============================] - 0s 1ms/step - loss: 4.4526e-04
Epoch 94/200
75/75 [==============================] - 0s 1ms/step - loss: 4.8661e-04
Epoch 95/200
75/75 [==============================] - 0s 1ms/step - loss: 4.6024e-04
Epoch 96/200
75/75 [==============================] - 0s 1ms/step - loss: 5.2222e-04
Epoch 97/200
75/75 [==============================] - 0s 1ms/step - loss: 4.3309e-04
Epoch 98/200
75/75 [==============================] - 0s 1ms/step - loss: 4.3518e-04
Epoch 99/200
75/75 [==============================] - 0s 1ms/step - loss: 4.3793e-04
Epoch 100/200
75/75 [==============================] - 0s 1ms/step - loss: 4.6133e-04
Epoch 101/200
75/75 [==============================] - 0s 1ms/step - loss: 3.

75/75 [==============================] - 0s 1ms/step - loss: 1.2223e-04
Epoch 186/200
75/75 [==============================] - 0s 1ms/step - loss: 1.2101e-04
Epoch 187/200
75/75 [==============================] - 0s 1ms/step - loss: 1.3550e-04
Epoch 188/200
75/75 [==============================] - 0s 1ms/step - loss: 1.2856e-04
Epoch 189/200
75/75 [==============================] - 0s 1ms/step - loss: 1.3674e-04
Epoch 190/200
75/75 [==============================] - 0s 1ms/step - loss: 1.6083e-04
Epoch 191/200
75/75 [==============================] - 0s 1ms/step - loss: 1.5386e-04
Epoch 192/200
75/75 [==============================] - 0s 1ms/step - loss: 1.5158e-04
Epoch 193/200
75/75 [==============================] - 0s 1ms/step - loss: 1.0262e-04
Epoch 194/200
75/75 [==============================] - 0s 1ms/step - loss: 1.1138e-04
Epoch 195/200
75/75 [==============================] - 0s 1ms/step - loss: 1.1024e-04
Epoch 196/200
75/75 [==============================] - 0s 1ms/step -

### Saving Models to be Called Elsewhere

In [7]:
AMCModel.save("AMCModel.h5")
GMEModel.save("GMEModel.h5")